In [1]:
import sys
sys.path.append('..')

In [2]:
from tqdm import tqdm

from lib.data import load_all_invoices, load_extracted_samples
from lib.ocr import OCRParams, ocr_sample
from lib.invoice import extract_invoice
from lib.text import extract_total

In [3]:
all_invoices = load_all_invoices()
len(all_invoices)

8411

In [4]:
# load samples
samples = [sample for sample in load_extracted_samples()]
len(samples)

8411

In [5]:
from lib.invoice import find_invoice_cached


invoice_dict = {}
for invoice in all_invoices:
    invoice_dict[invoice] = None
    
params = OCRParams(
    engine="easyocr",
    grayscale=False,
    threshold=None
)

detected_but_not_found = []

for sample in tqdm(samples):
    filename = sample["filename"][:-4] # .pdf
    
    text = sample["text"]
    if text == "":
        text = ocr_sample(sample, params, only_cache=True)
    
    invoice = find_invoice_cached(sample)
    total = extract_total(text)

    if invoice is not None and total is not None:
        if invoice in all_invoices:
            invoice_dict[invoice] = total
        else:
            # print(f'invoice detected {invoice} but not found in all_invoices')
            detected_but_not_found.append(invoice)

  1%|          | 87/8411 [00:16<25:32,  5.43it/s]  


KeyboardInterrupt: 

In [ ]:
len(detected_but_not_found)

In [ ]:
all_totals = [total for total in invoice_dict.values() if total is not None]
avg = sum(all_totals) / len(all_totals)
avg = round(avg, 2)
len(all_totals), len(all_totals) / len(samples), avg

In [ ]:
# write csv
with open("submission.csv", 'w') as f:
    f.write('Invoice Number,Total Charged\n')
    for invoice, total in invoice_dict.items():

        if total == None:
            total = 10341.62 # avg

        f.write(f'{invoice},{total}\n')